# Latent Semantic Analysis for Similarity Queries

Latent semantic analysis (LSA) is a technique in natural language processing, in particular distributional semantics, of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms. LSA assumes that words that are close in meaning will occur in similar pieces of text (the distributional hypothesis). A matrix containing word counts per paragraph (rows represent unique words and columns represent each paragraph) is constructed from a large piece of text and a mathematical technique called singular value decomposition (SVD) is used to reduce the number of rows while preserving the similarity structure among columns. Paragraphs are then compared by taking the cosine of the angle between the two vectors (or the dot product between the normalizations of the two vectors) formed by any two columns. Values close to 1 represent very similar words while values close to 0 represent very dissimilar words.

https://radimrehurek.com/gensim/tut1.html
https://radimrehurek.com/gensim/tut2.html
https://radimrehurek.com/gensim/tut3.html

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925

Seguir con:

https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html

In [0]:
# Imports

from google.colab import drive
drive.mount('/content/gdrive') # grant access to google drive filesystem

from pprint import pprint  # pretty-printer
from collections import defaultdict
import os
from gensim import corpora
from gensim import models
import unicodedata
import csv
import gensim 
from spacy.lang.es.stop_words import STOP_WORDS
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Initializations

# Add stop words in spanish
STOP_WORDS.add('y')
STOP_WORDS.add('a')
STOP_WORDS.add('o')
STOP_WORDS.add('publicidad')
STOP_WORDS.add('expansion')
STOP_WORDS.add('para')
STOP_WORDS.add('\n')

In [0]:
def createDictionary(publicacion):
    path = '/content/gdrive/My Drive/datos/'
    notesFile = u'all_notas_texto.csv'
    
    modeloNombre = publicacion # Possible values: 'Adn Político', 'Expansión', 'Life and Style', 'Manufactura', 'Obras', 'Quién', 'Todos'
    modeloArchivo = unicodedata.normalize('NFKD', publicacion.replace(" ", "").lower()).encode('ASCII', 'ignore').decode('unicode_escape') + '_model.txt'
    texts = []
    documents = []
    
    # Read text from notes from selected publication
    with open(os.path.join(path, notesFile)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        next(csv_reader) # skip headers
        if modeloNombre == 'Todos': 
            for row in csv_reader:
                # Elimina acentos, palabras de menos de dos caracteres y mayores de 15 caracteres
                documents.append(gensim.utils.simple_preprocess(row[4], deacc=True, min_len=2, max_len=15)) 
        else:
            for row in csv_reader: 
                if modeloNombre == row[2]:
                    documents.append(gensim.utils.simple_preprocess(row[4], deacc=True, min_len=2, max_len=15))
    
    # remove stop words
    for i in range(1, len(documents)):
        articulo = []
        for palabra in documents[i]:
            if palabra not in STOP_WORDS:
                articulo.append(palabra)
        documents[i] = articulo
    
    return documents





In [0]:
documents = createDictionary('Expansión')

In [0]:
type(documents)

list

In [0]:
path = '/content/gdrive/My Drive/datos/'
dictionary = corpora.Dictionary(documents)
dictionary.save(os.path.join(path, 'dictionary.dic'))  # store the dictionary, for future reference

corpus = [dictionary.doc2bow(text) for text in documents]
corpora.MmCorpus.serialize(os.path.join(path, 'vector.mm'), corpus)  # store to disk, for later use

In [0]:
dictionary.token2id

In [0]:
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=20, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=20, id2word=dictionary)
 

In [0]:
print("LDA Model:")
 
for idx in range(20):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 20))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(20):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.011*"mexico" + 0.007*"lee" + 0.006*"millones" + 0.006*"ano" + 0.005*"pesos" + 0.004*"nacional" + 0.004*"pemex" + 0.003*"presidente" + 0.003*"ciudad" + 0.003*"unidos" + 0.003*"empresa" + 0.003*"gobierno" + 0.003*"mdp" + 0.003*"anos" + 0.003*"jueves" + 0.002*"energia" + 0.002*"enero" + 0.002*"diciembre" + 0.002*"produccion" + 0.002*"reporte" + 0.002*"dolares" + 0.002*"peso" + 0.002*"comision" + 0.002*"precios" + 0.002*"recomendamos" + 0.002*"trump" + 0.002*"mexicana" + 0.002*"dolar" + 0.002*"lopez" + 0.002*"precio"
Topic #1: 0.010*"mexico" + 0.007*"empresas" + 0.006*"gobierno" + 0.005*"pemex" + 0.005*"lee" + 0.004*"millones" + 0.003*"ciudad" + 0.003*"anos" + 0.003*"ano" + 0.003*"empresa" + 0.003*"mercado" + 0.003*"sector" + 0.003*"datos" + 0.003*"leer" + 0.003*"presidente" + 0.003*"federal" + 0.003*"lopez" + 0.002*"nacional" + 0.002*"informacion" + 0.002*"obrador" + 0.002*"trabajadores" + 0.002*"mdp" + 0.002*"recursos" + 0.002*"pesos" + 0.002*"capital" + 0.002*"des

In [0]:
dictionary = corpora.Dictionary.load(os.path.join(path, 'dictionary.dic'))
corpus = corpora.MmCorpus(os.path.join(path, 'vector.mm'))  # comes from the first tutorial, "From strings to vectors"
print(corpus)

MmCorpus(3819 documents, 50018 features, 469587 non-zero entries)


In [0]:

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=500)

index = MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

#doc = "La cervecera Grupo Modelo, que pertenece a la belgobrasileña Ab InBev, anunció este martes que traerá cerca de 2,000 camiones eléctricos en los próximos tres años a México, como parte de sus objetivos para convertirse en una empresa 100% sustentable para 2025. Por ahora tenemos uno, esperamos traer otros tres más y los pondremos a prueba próximamente. Estamos buscando traer 2,000 de la mano con la empresa BYD. Por ahora estamos en el proceso de completarlos en los próximos tres años con estas unidades circulando en varias partes del país, principalmente en la Ciudad de México”, explica Tony Milikin, director de Adquisiciones y Sustentabilidad para AB InBev en entrevista con Expansión. Por ahora estamos en el proceso de completarlos en los próximos tres años con estas unidades circulando en varias partes del país, principalmente en la Ciudad de México”, explica Tony Milikin, director de Adquisiciones y Sustentabilidad para AB InBev en entrevista con Expansión. También están realizando esta iniciativa en Brasil en dónde buscan introducir 1,600 camiones eléctricos y en Estados Unidos lanzarán el próximo martes el primer auto de hidrógeno para circular. Gran parte de esta idea la retomaron de China, en dónde aproximadamente la mitad de los medios de transportes –entre ellos camiones, autobuses, taxis— son eléctricos. La parte más bonita es que están buscando en como cargarlos, pueden ser con paneles solares, eso es el futuro, todavía no estamos ahí pero vamos hacia allá. Creo que esto es un comienzo, agrega Milikin, cuya empresa busca reducir los gases invernaderos en un 25% para 2025. El camión tiene una autonomía estimada de 200 km y tarda en cargar dos horas y media, nuestra promedio es de 50 km con una carga puede cargara para cuatro días. "
#doc = "Garantizar la libre afiliación sindical y la transparencia de los contratos colectivos; combatir la violencia de género, proteger a los trabajadores migrantes y establecer mecanismos de cooperación internacional para cumplir las leyes laborales. Estos son los principales beneficios que traerá consigo la reforma laboral que discute a marchas forzadas el Congreso de la Unión, y que forma parte del nuevo Tratado entre México, Estados Unidos y Canadá (T-MEC, futuro sustituto del TLCAN), coincidieron varios especialistas consultados por Expansión."
#doc = "La inflación anual de México repuntó ligeramente en marzo, después de dos meses seguidos en desaceleración, debido a incrementos en los precios de la gasolina que fueron contrarrestados por disminuciones en los de algunos productos agrícolas. El índice de precios al consumidor avanzó en el periodo a un 4.0%, casi en línea con las expectativas del mercado, de acuerdo con datos del Instituto Nacional de Estadística y Geografía (INEGI) divulgados el martes."
#doc = " La elección extraordinaria del 2 de junio en Puebla es de dos, aseguró la dirigencia del Partido Acción Nacional (PAN) a la par que reconoció una desventaja de 17 puntos de su candidato Enrique Cárdenas frente a Miguel Barbosa Huerta, de Morena."
#doc = "La Comisión Federal de Electricidad (CFE) comprará a la multinacional suiza Glencore cerca de 4.9 millones de toneladas de carbón térmico para generar electricidad en sus plantas, en 12 contratos que suman un valor de 519.6 millones de dólares (mdd)."
#doc = "Pero la transnacional suiza ofreció los mejores precios para cada una de las partidas donde compitió, por lo que la CFE adquirirá el carbón por un precio que irá de los 22 dólares por Giga Caloría a los 16.75 dólares, según datos del fallo publicado este martes. El combustible servirá para suministrar a la planta conocida como Petacalco, ubicada en Guerrero."
doc = "La influencia incomparable de la Reserva Federal (Fed) sobre la economía mundial se debe en gran parte a la creencia de que simplemente no le importa la política. El banco central estadounidense ha demostrado que no tiene miedo de aplicarle freno a la economía para evitar una inflación descontrolada, incluso si hay una elección a la vuelta de la esquina. Ese historial crea estabilidad en los mercados globales y da confianza a los inversores. Ahora, el presidente de Estados Unidos, Donald Trump, está poniendo en riesgo la percepción cuidadosamente construida de independencia de la Fed. El mandatario ha ataca repetidamente a la Reserva e incluso ha dicho que consideraba despedir a su presidente, Jerome Powell, a quien él mismo eligió."
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space

index.save(os.path.join(path, 'dictionary.index'))
index = MatrixSimilarity.load(os.path.join(path, 'dictionary.index'))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
print(vec_lsi)

[(0, 0.6774211369617194), (1, 0.21800966828838841), (2, 0.14021105597386163), (3, -0.03994364865017315), (4, -0.06608743704782812), (5, -0.21100717536854033), (6, -0.05987696052143764), (7, 0.06241315237170016), (8, -0.2194505506132475), (9, -0.0395311124726061), (10, 0.18962070628951957), (11, 0.3556848469100659), (12, -0.08453453470883023), (13, 0.6399732555056578), (14, -0.04043910316784678), (15, -0.20254510302364973), (16, 0.20783393095621197), (17, 0.08990521644136569), (18, -0.164190296922661), (19, -0.2985372955584372), (20, 0.014884089894890548), (21, 0.05707937920302826), (22, -0.12163788844627427), (23, -0.12495172330776742), (24, 0.31383299065458675), (25, -0.008851287815788223), (26, -0.21535387982886228), (27, -0.0022311411487147986), (28, 0.10636968136582163), (29, 0.1709262922767366), (30, 0.06708804820974368), (31, 0.1540776168913443), (32, 0.26548729219942835), (33, -0.15381634382088258), (34, -0.0338272435776206), (35, -0.26246756638808505), (36, 0.15721924579813767)

In [0]:
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i in sims:
    print(f'{i[1]:1.3f}', documents[i[0]])

0.540 ['washington', 'expresidenta', 'reserva', 'federal', 'unidos', 'janet', 'yellen', 'aseguro', 'lunes', 'presidente', 'eu', 'donald', 'trump', 'falta', 'entendimiento', 'politica', 'economica', 'mision', 'banco', 'central', 'constantes', 'criticas', 'organismo', 'dudo', 'trump', 'capaz', 'metas', 'fed', 'maximo', 'estabilidad', 'precios', 'objetivos', 'asignados', 'fed', 'congreso', 'yellen', 'cuestionada', 'consideraba', 'trump', 'comprende', 'politica', 'economica', 'cuestionado', 'martes', 'senador', 'democrata', 'sherrod', 'brown', 'actual', 'presidente', 'fed', 'jerome', 'powell', 'quiso', 'desmarcarse', 'predecesora', 'cargo', 'comentaria', 'afirmacion', 'yellen', 'presidente', 'yellen', 'designada', 'expresidente', 'barack', 'obama', 'mujer', 'frente', 'banco', 'central', 'estadounidense', 'reemplazada', 'iniciativa', 'trump', 'llegase', 'casa', 'blanca']
0.511 ['funcionarios', 'reserva', 'federal', 'fed', 'consideraron', 'pausar', 'alzas', 'tasas', 'interes', 'mes', 'plante

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 ['ciudad', 'mexico', 'cnn', 'ano', 'autos', 'lujo', 'land', 'rover', 'jaguar', 'volvo', 'tuvieron', 'aumentos', 'significativos', 'ventas', 'anuales', 'reporte', 'asociacion', 'mexicana', 'industria', 'automotriz', 'amia', 'emitido', 'martes', 'anotar', 'repuntes', 'comparacion', 'caso', 'marca', 'land', 'rover', 'jaguar', 'aumento', 'ventas', 'registrar', 'incremento', 'anual', 'seguido', 'volvo', 'lincoln', 'bmw', 'mercedes', 'benz', 'aumento', 'ventas', 'anuales', 'lee', 'distribuidores', 'piden', 'devolucion', 'saldos', 'favor', 'iva', 'vendedores', 'autos', 'marcas', 'regular', 'ano', 'sonrio', 'lujo', 'volkswagen', 'vio', 'caidas', 'ventas', 'seguida', 'ford', 'nissan', 'motors', 'renault', 'ventas', 'ano', 'ventas', 'ligeras', 'reporte', 'amia', 'divulga', 'cifras', 'publicadas', 'instituto', 'nacional', 'estadistica', 'geografia', 'inegi', 'venta', 'vehiculos', 'ligeros', 'diciembre', 'menor', 'unidades', 'comercializadas', 'mes', 'diciembre', 'ano', 'vendieron', 'unidades', '